# Denoising Diffusion Probababilistic Model

This tutorial will illustrate how you can train a diffusion model on your dataset. Many downstream applications of diffusion models involve training an unconditional diffusion first, before using such a model in a specialized prediction scheme, such as inpainting, denoising etc. Therefore, we will first show how to train an unconditional model and in a separate tutorial available [here], you can see an example of inpainting that also aims to illustrate what role UQ can potentially play.

## Theoretical Background

Numerous great introductions to diffusion models exist (linked below), so therefore we will only try to give a brief overview and intuition about this class of models.

## Imports

In [8]:
from torchgeo.transforms import AugmentationSequential
import kornia.augmentation as K
from lightning import LightningDataModule, Trainer
from torch.utils.data import DataLoader
from torchvision.datasets import VisionDataset
from typing import Any
from glob import glob
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor

from denoising_diffusion_pytorch.denoising_diffusion_pytorch import Unet, GaussianDiffusion

from lightning_uq_box.uq_methods import DDPM


## Implementation

Instead of implementing these models from scratch, we will utilize the implementations of this great [repo](https://github.com/lucidrains/denoising-diffusion-pytorch) by [lucidrains](https://lucidrains.github.io/), so please consider giving it a star to show your appreciation. The Lightning-UQ-Box provides LightningModule wrappers that execute the training via Lightning so you can benefit from all the multi-gpu and logging support that Lightning provides.

For the model we need three parts:
- the denoising model architecture, commonly a Unet
- the diffusion model that can execute the denoising step as well as sampling
- the diffusion LightningModule that can execute the training via lightning

The first two components are implemented in [denoising-diffusion-pytorch](https://github.com/lucidrains/denoising-diffusion-pytorch) and the last one in the Lightning-UQ-Box.

In addition, we need a LightningDataModule that loads the images for our task.

### Unet

In [2]:
unet = Unet(
    dim=64,
    dim_mults=[1, 2, 4, 8, 16, 32],
    channels=3,
    self_condition=True
)

### Diffusion Model

In [3]:
diff_model = GaussianDiffusion(
    unet,
    image_size= 224,
    timesteps= 500,
    sampling_timesteps=250,
    beta_schedule="cosine"
)

**NOTE**

The more recent paper by [Karras et al 2022]() proposed some changes that can improve results. If you want to use that version, use `ElucidatedDiffusion` instead of `GaussianDiffusion` availabel as `from denoising_diffusion_pytorch.elucidated_diffusion import ElucidatedDiffusion`. For this implementation you also need to change the Unet architecture to either use `learned_sinusoidal_cond=True` or `random_fourier_features=True`.

### LightningModule

In [4]:
ddpm = DDPM(
    diff_model,
    log_samples_every_n_steps=200
)

### DataModule

For our purposes we will used the imagery of the [Million-Aid Dataset](https://captain-whu.github.io/DiRS/) that can also be loaded with [torchgeo](https://github.com/microsoft/torchgeo). The goal will be to train a model that can generate samples from the training data distribution.

In [6]:
class MillionAidDataset(VisionDataset):
    resize = K.Resize((224, 224))
    def __init__(self, root: str):
        super().__init__(root)

        self.fpaths = sorted(glob(root + '/*.jpg', recursive=True))
        assert len(self.fpaths) > 0, "File list is empty. Check the root."

    def __len__(self):
        return len(self.fpaths)

    def __getitem__(self, index: int):
        fpath = self.fpaths[index]
        img = Image.open(fpath).convert('RGB')


        img = pil_to_tensor(img) / 255.
        img = (img).squeeze(0)

        return {"input": img}


class MillionAIDDataModule(LightningDataModule):
    """DataModule for MillionAID dataset."""

    def __init__(self, root: str, batch_size: int = 64, num_workers: int = 4):
        """Initialize a new instance of MillionAIDDataModule.
        
        Args:
            root: The root directory of the dataset.
            batch_size: The batch size.
            num_workers: The number of workers for the DataLoader.
        """
        super().__init__()

        self.root = root
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.aug = AugmentationSequential(
            K.RandomHorizontalFlip(p=0.5),
            K.RandomVerticalFlip(p=0.5),
            data_keys=["input"],
        )

    def setup(self, stage: str | None = None):
        self.train = MillionAidDataset(root=self.root)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def on_after_batch_transfer(self, batch: dict[str, Any], dataloader_idx: int) -> dict[str, Any]:
        """Apply augmentations to the batch."""
        return self.aug(batch)

## Training

Training Diffusion Models is quiet computationally expensive. We have therefore pretrained a model to save a lot of time. However, the code below shows how you can train a model on your own dataset. In the included gif, you can see how the model capability to generate samples increases over training time, first random blobs and later samples that look like data points from our training distribution, namely the MillionAid dataset.

TODO: Include GIF here

In [9]:
LOAD_CKPT = True

In [7]:
dm = MillionAIDDataModule(root="/mnt/SSD2/nils/ocean_bench_exps/diffusion/data/million/test/test", batch_size=64, num_workers=4)

In [ ]:
if not LOAD_CKPT:
    trainer = Trainer(
        accelerator="gpu",
        strategy="ddp",
        accumulate_grad_batches=4,
        devices=[4, 5, 6, 7],
        min_epochs=800,
        max_epochs=1000,
        log_every_n_steps=50,
        enable_progress_bar=True,
        enable_checkpointing=True
    )
    trainer.fit(ddpm, dm)

else:
    # load checkpoint

## Sampling

In the following sections we will generate some samples from the diffusion model.